# dependencias

In [1]:
import igraph as ig
import json
import numpy as np
import sys
import os 
if '../src' not in sys.path:
    sys.path.append('../src')
import utilidadest5 as ut

# lectura de los datos 

In [2]:
# leemos el json
# Estadística,Biología, Farmacia, Física, Geociencias, Matemáticas y Química.
facultades = ['Estadística', 'Biología', 'Farmacia', 'Física', 'Geociencias', 'Matemáticas', 'Química']
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Estadística.json') as f:
    data_est = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Biología.json') as f:
    data_bio = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Farmacia.json') as f:
    data_far = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Física.json') as f:
    data_fis = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Geociencias.json') as f:
    data_geo = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Matemáticas.json') as f:
    data_mat = json.load(f)
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Química.json') as f:
    data_qui = json.load(f)    

data = [data_est, data_bio, data_far, data_fis, data_geo, data_mat, data_qui]

In [3]:
# nodos 
nodos = [nodos['nodes'] for nodos in data]
# aristas
aristas = [aristas['links'] for aristas in data]
# info aparte 
options = [options['options'] for options in data]

In [4]:
id_evitar = [nodo[0]['id'] for nodo in nodos]
id_evitar

['2d4d5a17-7ef0-4fc4-8b25-d1af93445f50',
 'e0a9295b-404c-4134-99f0-756e8849413a',
 'a43e06fd-60e5-48c6-8a38-ed271b978ea4',
 '425bda01-002d-4b05-95fb-71196da8736e',
 'f9f44c87-afce-4bd3-a5aa-4988976fa40b',
 '9b94b0ad-bff6-4f20-ae80-170d728a6b9c',
 'c290f8c7-1919-44c2-8e05-111249363db7']

In [5]:
# Estadística,Biología, Farmacia, Física, Geociencias, Matemáticas y Química.
G_est= ut.grafo_biparito_t5(nodos[0], aristas[0],id_evitar[0])
G_bio= ut.grafo_biparito_t5(nodos[1], aristas[1],id_evitar[1])
G_far= ut.grafo_biparito_t5(nodos[2], aristas[2],id_evitar[2])
G_fis= ut.grafo_biparito_t5(nodos[3], aristas[3],id_evitar[3])
G_geo= ut.grafo_biparito_t5(nodos[4], aristas[4],id_evitar[4])
G_mat= ut.grafo_biparito_t5(nodos[5], aristas[5],id_evitar[5])
G_qui= ut.grafo_biparito_t5(nodos[6], aristas[6],id_evitar[6])
Grafos = [G_est, G_bio, G_far, G_fis, G_geo, G_mat, G_qui]

In [6]:
adyacencia = [] 
autores_list = []
for G in Grafos:
    # matriz de adyacencia
    A = np.array(G.get_biadjacency()[0]) 
    autores = G.vs.select(type=False)['label']
    autores_list.append(autores)
    print(len(autores))
    # A * A^t
    A_A_t = np.dot(A, A.T)
    # remplazamos la diagonal con ceros 
    np.fill_diagonal(A_A_t, 0)
    adyacencia.append(A_A_t)
    

58
108
90
146
82
192
210


In [7]:
grafos_autores = []
for i in range(len(adyacencia)):
    grafos_autores.append(ut.grafo_matriz_adyacencia_t5(adyacencia[i], autores_list[i]))

In [8]:
for grafo_autores in grafos_autores:
    print(grafo_autores.summary())

IGRAPH U-W- 58 706 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 108 1584 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 90 1031 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 146 2607 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 82 510 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 192 2148 -- 
+ attr: label (v), weight (e)
IGRAPH U-W- 210 6237 -- 
+ attr: label (v), weight (e)


In [9]:
for i in range(len(grafos_autores)):
    layout = grafos_autores[i].layout("fr")
    layout.scale(1.0)
    # tranparencia de los nodos
    grafos_autores[i].vs['color'] =  "#00BFFF80"
    
    ig.plot(grafos_autores[i], layout = layout, bbox = (1000, 1000), margin = 20,vertex_label= None,vertex_size=15 ).save('../output/taller 5_plot/grafo_autores_'+ facultades[i] +'.png')


In [10]:
# estadisticas basicas 
estadisticas = []
for i in range(len(grafos_autores)):
    A =ut.estadisticas_descriptivas_t5(grafos_autores[i])
    A['facultad'] = facultades[i]
    estadisticas.append(A)

In [11]:
import pandas as pd
estadisticas = pd.DataFrame(estadisticas)
estadisticas1 =estadisticas[['']]

,diametro,nodo_mayor_grado,mayor_grado,nodo_max_intermediacion,nodo_max_cercania,nodo_max_eigen,grado_promedio,clan_mas_grande,densidad_red,transitividad_global,facultad
0,3,[Vanegas Penagos Luis Hernando],40,Vanegas Penagos Luis Hernando,Vanegas Penagos Luis Hernando,Vanegas Penagos Luis Hernando,24.344828,36,0.427102,0.925529,Estadística
1,2,"[Caro Quintero Alejandro, Muñoz Duran Joao Vic...",60,Caro Quintero Alejandro,Nuñez Del Castillo Fernando Alfonso,Caro Quintero Alejandro,29.333333,53,0.274143,0.936705,Biología
2,3,"[Ospina Giraldo Luis Fernando, Pinzon Serrano ...",47,Vallejo Diaz Bibiana Margarita Rosa,Ospina Giraldo Luis Fernando,Marin Loaiza Juan Camilo,22.911111,42,0.257428,0.930438,Farmacia
3,3,[Dussan Cuenca Anderson],76,Perilla Perilla Carlos Joel,Cediel Casas Gilberto,Dussan Cuenca Anderson,35.712329,69,0.246292,0.957474,Física
4,2,[Sarmiento Perez Gustavo Adolfo],36,Sarmiento Perez Gustavo Adolfo,Franco Latorre Rodolfo De Jesus,Sarmiento Perez Gustavo Adolfo,12.439024,25,0.153568,0.862896,Geociencias
5,5,"[Nuñez Alarcon Daniel, Rojas Santana Edixon Ma...",71,Gomez Sierra Cesar Augusto,Ramirez Vallejo Jorge Eduardo,Nuñez Alarcon Daniel,22.375000,56,0.117147,0.872746,Matemáticas
6,3,[Alí Torres Jorge Isaac],115,Chegwin Angarita Carolina,Alí Torres Jorge Isaac,Alí Torres Jorge Isaac,59.400000,106,0.284211,0.972025,Química


In [16]:
estadisticas.to_latex('../output/estadisticas_autores.tex', index = False)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\bcriv\miniconda3\envs\semillero\Lib\site-packages\pandas\compat\_optional.py", line 135, in import_optional_dependency
    module = importlib.import_module(name)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bcriv\miniconda3\envs\semillero\Lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1140, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'tabulate'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\bcriv\miniconda3\envs\semillero\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, s

In [13]:
# subgrafo conexo
grafos_autores_conexo = []
for grafo_autores in grafos_autores:
    G_autores_conexo = grafo_autores.connected_components().giant()
    grafos_autores_conexo.append(G_autores_conexo)
# graficamos
for i in range(len(grafos_autores_conexo)):
    layout = grafos_autores_conexo[i].layout("fr")
    layout.scale(1.0)
    # tranparencia de los nodos
    grafos_autores_conexo[i].vs['color'] =  "#00BFFF80"
    
    ig.plot(grafos_autores_conexo[i], layout = layout, bbox = (1000, 1000), margin = 20,vertex_label= None,vertex_size=15 ).save('../output/taller 5_plot/grafo_autores_conexos_'+ facultades[i] +'.png')


In [12]:
sum_pesos_aristas = G_autores_conexo.strength(weights='weight')

# Normalizar la suma de los pesos de las aristas para ajustarla al rango de tamaños de los nodos
max_sum_pesos_aristas = max(sum_pesos_aristas)
sizes = [x / max_sum_pesos_aristas * 40 for x in sum_pesos_aristas]  # Ajusta el rango de tamaño deseado
G_autores_conexo.vs["size"] = sizes

In [14]:
for i in range(len(grafos_autores_conexo)): 
    dendrogram = grafos_autores_conexo[i].community_edge_betweenness(weights="weight")
    layout_autores_conexo = G_autores_conexo[i].layout("kk")

    # Obtener las comunidades finales
    communities = dendrogram.as_clustering()

    # Obtener el número de comunidades
    num_communities = len(communities)

    # Asignar colores a cada comunidad
    palette = ig.RainbowPalette(n=num_communities)
    community_colors = [palette.get(i) for i in communities.membership]


    # Definir transparencia para los nodos
    node_transparency = 0.5  # Valor entre 0 y 1, donde 0 es completamente transparente y 1 es completamente opaco

    # Convertir los colores de las comunidades a RGBA con la transparencia deseada
    community_colors_with_alpha = [color[:-1] + (node_transparency,) for color in community_colors]

    # Dibujar el grafo con nodos semi-transparentes
    ig.plot(grafos_autores_conexo[i],'../output/taller4_plot/caso_6/comunidades_6.png', layout=layout_autores_conexo,vertex_color=community_colors_with_alpha,vertex_label= None,).save('../output/taller 5_plot/comunidades_autores_conexo.png')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\bcriv\miniconda3\envs\semillero\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\bcriv\AppData\Local\Temp\ipykernel_5132\1700992295.py", line 3, in <module>
    layout_autores_conexo = G_autores_conexo[i].layout("kk")
                            ~~~~~~~~~~~~~~~~^^^
KeyError: 'Attribute does not exist'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\bcriv\miniconda3\envs\semillero\Lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bcriv\miniconda3\envs\semillero\Lib\site-packages\IPython\core\ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^